# Notebook to review the pickles files

## Import the needed libraries

In [1]:
import pandas as pd
import os
from itertools import chain
import glob

## Review the prediction pickles. 

### Bunches per row (Racimos por hilera)

Get the number of "racimos" per "hilera". We use just the the prediction pickles to get this values

In [21]:
pickles = glob.glob(PICKLE_DIR+"*.pkl")
pickles

df = pd.read_pickle(pickles[0])

df = pd.DataFrame(df, columns = ["campo_id","cuartel_id","hilera_id","am_pm","racimo_id","racimo_area"])
df


,campo_id,cuartel_id,hilera_id,am_pm,racimo_id,racimo_area
0,1,1,10,1,1,6726
1,1,1,10,1,1,7200
2,1,1,10,1,1,7380
3,1,1,10,1,1,8190
4,1,1,10,1,1,7714
...,...,...,...,...,...,...
80,1,1,10,1,72,12528
81,1,1,10,1,72,9855
82,1,1,10,1,72,8512
83,1,1,10,1,72,6930


In [27]:
PICKLE_DIR = '../../stuff/pickles/prediction_pickles/'

#List to stack the dframes
df_frames = []

#Open each pickle file and append to a list
for file in glob.glob(PICKLE_DIR+"*.pkl"):
    
    #Read pickle and transform to a data frame
    df = pd.read_pickle(file)
    df = pd.DataFrame(df, columns = ["campo_id","cuartel_id","hilera_id","am_pm","racimo_id","racimo_area"])
    
    #Append to the list
    df_frames.append(df)

#Create a data frame of all the pickles
full_df = pd.concat(df_frames)

#Conteo de racimos por hilera
racimos_hilera = full_df[["hilera_id","racimo_id"]].groupby(["hilera_id"]).count().reset_index()
racimos_hilera["hilera_id"] = pd.to_numeric(racimos_hilera["hilera_id"]) #transform to numeric column

#Sort by hilera
sorted_hilera = racimos_hilera.sort_values(by=["hilera_id"])
sorted_hilera = sorted_hilera.rename(columns={'racimo_id': 'num_racimos'})

#Saved the .csv file
#sorted_hilera.to_csv("racimos_por_hilera.csv")
#df_output.to_csv("prediction.csv")

sorted_hilera.head()
#full_df.head()
    
    

,hilera_id,num_racimos
0,10,170


### Create Mega Pickle


In [16]:
PICKLE_DIR = '../../stuff/pickles/TestCampo/prediction_pickles/'
OUTPUT_DIR = "../../stuff/pickles/TestCampo/"

#List to stack the dframes
df_frames = []

#Open each pickle file and append to a list
for file in glob.glob(PICKLE_DIR+"*.pkl"):
    
    #Read pickle and transform to a data frame
    df = pd.read_pickle(file)
    df = pd.DataFrame(df, columns = ["campo_id","cuartel_id","hilera_id","am/pm","racimo_id","racimo_area"])
    
    #Filter the repeated racimos, only chose the first ocurrence
    df = df.groupby(["racimo_id"]).first().reset_index()
    
    #Append to the list
    df_frames.append(df)

#Create a data frame of all the pickles
full_df = pd.concat(df_frames).reset_index()

#Rename and store the new pickle
df_output = full_df.rename(columns= {i : str(i).upper() for i in list(filter_df.columns)})
df_output = df_output[["CAMPO_ID","CUARTEL_ID","HILERA_ID","AM/PM","RACIMO_ID","RACIMO_AREA"]]
df_output

print("GUARDANDO DATAFRAME A PICKLE...")
#df_output.to_pickle(OUTPUT_DIR+"mega_pickle.pkl")

full_df
#filter_df

GUARDANDO DATAFRAME A PICKLE...


,index,racimo_id,campo_id,cuartel_id,hilera_id,am/pm,racimo_area
0,0,1,2,2,36,1,33390
1,1,2,2,2,36,1,12629
2,2,3,2,2,36,1,16600
3,3,7,2,2,36,1,9035
4,4,14,2,2,36,1,8536
...,...,...,...,...,...,...,...
1984,1,12,2,2,10,1,9718
1985,2,19,2,2,10,1,17864
1986,3,32,2,2,10,1,7980
1987,4,35,2,2,10,1,6039


In [7]:
#Checking the resulting mega pickle
df = pd.read_pickle("../../stuff/pickles/TestCampo/mega_pickle.pkl")
df.groupby("AM/PM").first()

,CAMPO_ID,CUARTEL_ID,HILERA_ID,RACIMO_ID,RACIMO_AREA
AM/PM,,,,,
2,2,2,36,1,17484


In [9]:
df = pd.read_pickle("../../stuff/pickles/TestCampo/prediction_pickles/prediction_2_2_36_1.pkl")
df = pd.DataFrame(df, columns = ["campo_id","cuartel_id","hilera_id","am/pm","racimo_id","racimo_area"])
df

,campo_id,cuartel_id,hilera_id,am/pm,racimo_id,racimo_area
0,2,2,36,1,1,33390
1,2,2,36,1,2,12629
2,2,2,36,1,3,16600
3,2,2,36,1,1,28890
4,2,2,36,1,2,12474
...,...,...,...,...,...,...
33187,2,2,36,1,8271,9453
33188,2,2,36,1,8271,10184
33189,2,2,36,1,8271,9514
33190,2,2,36,1,8271,9344


### Review the locations pickles

Get the dictionaries of location, where each value in the dictionary has:

- key: id_racimo
- value: (frame_count,x_center)

In [29]:
!ls ../../stuff/pickles/location_pickles  

locations_1_1_10_1.pkl	locations_1_1_10_2.pkl	vinaSP


In [31]:
def location_pickle_to_df(file_name):
    #get the hilera id
    n_hilera = file_name.split("_")[1]
    
    #Read the pickle as dict
    df = pd.read_pickle(os.path.join(PICKLE_DIR,file_name))
    
    #Transform to pickle to dataframe
    df = pd.DataFrame(df.items(), columns=['id_racimo', 'info_racimo'])
    df['hilera_id'] = n_hilera
    
    #Split the info racimo and concat in a new data frame
    tmp = pd.DataFrame(df['info_racimo'].tolist(), index=df.index, columns=['frame_count','x_center'])
    df = pd.concat([df['hilera_id'],df['id_racimo'],tmp],axis=1)
    
    return df
    

In [32]:
PICKLE_DIR = '../../stuff/pickles/location_pickles/'

dframes = []

#Open each pickle file and append to a list
for file_name in os.listdir(PICKLE_DIR):
    if file_name.endswith(".pkl"):
        print("Opening file: ",file_name)
        df = location_pickle_to_df(file_name)

        dframes.append(df)
 
#concat all the dataframes
full_df = pd.concat(dframes)

#Saved the .csv file
#full_df.to_csv("localtions.csv")

full_df

Opening file:  locations_1_1_10_1.pkl
Opening file:  locations_1_1_10_2.pkl


,hilera_id,id_racimo,frame_count,x_center
0,1,1,5,32.5
1,1,12,33,43.0
2,1,19,49,66.5
3,1,32,80,78.5
4,1,35,87,39.5
5,1,72,136,37.5
0,1,1,5,32.5
1,1,12,33,43.0
2,1,19,49,66.5
3,1,32,80,78.5
